In [1]:
import warnings
import sys
import pathlib
import torch
import gpytorch
from IPython.display import HTML


warnings.filterwarnings('ignore')
sys.path.insert(0, str(pathlib.Path.cwd()/"src"))
%load_ext autoreload
%autoreload 2

import loaders.hrrr
import fields.continuous

In [2]:
import loaders.hrrr
import fields.discrete
import fields.continuous
import models.gp
import models.neural_flow

In [3]:
hrrr = loaders.hrrr.Data(date="2024-09-26 17:00", hours=3, level= 700)

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-26 17:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-26 17:00 UTC F01 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-26 17:00 UTC F02 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-26 17:00 UTC F03 ┊ GRIB2 @ aws ┊ IDX @ aws


In [4]:
#extent=(-85.5, -75.1, 30.5, 36.5)


In [5]:
dvf = hrrr.vector_field(extent=(-87.5,-81.5,24.5,29))
dsf = hrrr.scalar_field(extent=(-87.5,-81.5,24.5,29))

In [6]:
#HTML(dvf.plot().to_html5_video())

In [7]:
#HTML(dsf.plot().to_html5_video())

In [ ]:
dsf.coord_field.TXY.shape[0]/4

2450.0

In [11]:
dsf.coord_field.TXY_scaled

tensor([[ 0.0000, -0.6667,  0.6806],
        [ 0.0000, -0.6389,  0.6806],
        [ 0.0000, -0.6111,  0.6806],
        ...,
        [ 1.0000,  0.6111, -0.6806],
        [ 1.0000,  0.6389, -0.6806],
        [ 1.0000,  0.6667, -0.6806]])

In [106]:
import gc 
gc.collect()
torch.cuda.empty_cache()

In [114]:
cvf = fields.continuous.VectorField()

In [115]:
cvf.train_gp(dvf, dsf, batch_size = 2450, epochs=100)

Epoch 1/100 — rmse: 36.0606 - ll: 0.3695 - mean: -0.1000 - sigma2: 0.7444 - tau2: 0.0909 - l0: 2.0000 - l1: 0.4618 - l2: 0.4618


Minibatch:   0%|          | 0/1 [00:00<?, ?it/s]

Epoch 10/100 — rmse: 36.0606 - ll: 0.1684 - mean: -0.8721 - sigma2: 1.2823 - tau2: 0.0556 - l0: 2.0000 - l1: 0.2139 - l2: 0.2222


Epoch 20/100 — rmse: 36.0606 - ll: 0.1229 - mean: -0.6826 - sigma2: 1.7320 - tau2: 0.0359 - l0: 2.0000 - l1: 0.1033 - l2: 0.1288


Epoch 30/100 — rmse: 36.0606 - ll: 0.1166 - mean: -0.2024 - sigma2: 1.5106 - tau2: 0.0379 - l0: 2.0000 - l1: 0.1172 - l2: 0.1323


...

Epoch 60/100 — rmse: 36.0606 - ll: 0.1056 - mean: -0.3543 - sigma2: 0.9045 - tau2: 0.0401 - l0: 2.0000 - l1: 0.1119 - l2: 0.1262


Epoch 70/100 — rmse: 36.0606 - ll: 0.1036 - mean: -0.4223 - sigma2: 0.7103 - tau2: 0.0411 - l0: 2.0000 - l1: 0.1078 - l2: 0.1230


Epoch 80/100 — rmse: 36.0606 - ll: 0.1026 - mean: -0.3750 - sigma2: 0.5841 - tau2: 0.0410 - l0: 2.0000 - l1: 0.1013 - l2: 0.1183


Epoch 90/100 — rmse: 36.0606 - ll: 0.1023 - mean: -0.3493 - sigma2: 0.5289 - tau2: 0.0397 - l0: 2.0000 - l1: 0.0964 - l2: 0.1094


Epoch 100/100 — rmse: 36.0606 - ll: 0.1023 - mean: -0.3522 - sigma2: 0.5052 - tau2: 0.0398 - l0: 2.0000 - l1: 0.0948 - l2: 0.1091


In [116]:
cvf.train_flow(dvf, dsf, batch_size=4000, epochs=150)

Epoch 1/150 — rmse: 27.2632 - ll: 0.9879 - mean: -0.3522 - sigma2: 0.5052 - tau2: 0.0398 - l0: 2.0000 - l1: 0.0948 - l2: 0.1091


Epoch 10/150 — rmse: 19.6139 - ll: 0.7136 - mean: -0.3522 - sigma2: 0.5053 - tau2: 0.0398 - l0: 1.9999 - l1: 0.0948 - l2: 0.1091


Epoch 20/150 — rmse: 15.8702 - ll: 0.6504 - mean: -0.3522 - sigma2: 0.5053 - tau2: 0.0399 - l0: 1.9999 - l1: 0.0947 - l2: 0.1091


Epoch 30/150 — rmse: 13.6859 - ll: 0.6312 - mean: -0.3522 - sigma2: 0.5053 - tau2: 0.0399 - l0: 1.9999 - l1: 0.0947 - l2: 0.1090


Epoch 40/150 — rmse: 12.0268 - ll: 0.5999 - mean: -0.3522 - sigma2: 0.5053 - tau2: 0.0399 - l0: 1.9999 - l1: 0.0947 - l2: 0.1090


...

Epoch 110/150 — rmse: 6.4342 - ll: 0.4914 - mean: -0.3523 - sigma2: 0.5053 - tau2: 0.0399 - l0: 1.9999 - l1: 0.0947 - l2: 0.1090


Epoch 120/150 — rmse: 6.1352 - ll: 0.4900 - mean: -0.3523 - sigma2: 0.5053 - tau2: 0.0399 - l0: 1.9999 - l1: 0.0947 - l2: 0.1090


Epoch 130/150 — rmse: 5.9587 - ll: 0.4850 - mean: -0.3523 - sigma2: 0.5053 - tau2: 0.0399 - l0: 1.9999 - l1: 0.0947 - l2: 0.1090


Epoch 140/150 — rmse: 5.9604 - ll: 0.4870 - mean: -0.3523 - sigma2: 0.5053 - tau2: 0.0399 - l0: 1.9999 - l1: 0.0947 - l2: 0.1090


Epoch 150/150 — rmse: 6.2051 - ll: 0.4854 - mean: -0.3523 - sigma2: 0.5053 - tau2: 0.0399 - l0: 1.9999 - l1: 0.0947 - l2: 0.1090


In [117]:
HTML(cvf.plot(dvf.coord_field).to_html5_video())

In [78]:
dvf.RMS

tensor(28.6544)

In [122]:
dsf.coord_field.TXY_scaled

tensor([[ 0.0000, -0.6667,  0.6806],
        [ 0.0000, -0.6389,  0.6806],
        [ 0.0000, -0.6111,  0.6806],
        ...,
        [ 1.0000,  0.6111, -0.6806],
        [ 1.0000,  0.6389, -0.6806],
        [ 1.0000,  0.6667, -0.6806]])